In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind


In [2]:
biom=pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Biomarkers in CSF\Biomarkers.xlsx", header=0, index_col='Gene names'
             )
biom.head()

,ALS,ALS.1,ALS.2,ALS.3,Control,ALS.4,ALS.5,ALS.6,Control.1,Control.2,ALS.7,ALS.8,Control.3,Control.4
Gene names,,,,,,,,,,,,,,
UHRF1BP1L,1.549,1.083,0.907,1.284,0.925,0.647,0.620,0.794,1.109,0.673,NaN,NaN,NaN,NaN
IGLC7,1.002,0.577,0.635,2.081,0.827,NaN,NaN,NaN,NaN,NaN,0.910,0.841,2.379,0.706
SSC5D,1.071,1.009,1.021,0.900,0.990,1.055,1.023,0.991,1.098,0.942,1.046,0.923,0.911,0.903
MEGF11,1.141,0.867,0.947,0.500,1.040,0.818,1.044,0.958,1.050,0.836,NaN,NaN,NaN,NaN
IGLON5,0.952,0.860,1.158,0.554,1.030,1.050,1.134,1.135,1.014,1.064,0.919,0.791,1.245,1.282


In [3]:
biom.shape

(1530, 14)

In [4]:
# Columns belonging to Group 1 and 2
ALS_columns = [col for col in biom.columns if "ALS" in col]
Control_columns = [col for col in biom.columns if "Control" in col]


# Filter rows where at least 50% of Group 1 values are present, do the same for Group 2
filtered_als = biom[biom[ALS_columns].notna().mean(axis=1) >= 0.5]
cont_filter = filtered_als[filtered_als[Control_columns].notna().mean(axis=1) >= 0.5]

In [5]:
cont_filter.shape

(1147, 14)

In [7]:
#cont_filter.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Biomarkers in CSF\Biomarkers drop NA rows.xlsx")

In [6]:
cont_filt= pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Biomarkers in CSF\Biomarkers drop NA rows.xlsx", header=0, index_col='Gene names')

In [9]:
cont_filt.shape

(1140, 14)

In [7]:
# Automatically identify group columns by their prefixes
als = [col for col in cont_filt.columns if col.startswith('ALS.')]
control = [col for col in cont_filt.columns if col.startswith('Control.')]

# Perform t-tests row-wise
results = []
for i, row in cont_filt.iterrows():
    t_stat, p_value = ttest_ind(row[als].dropna(), row[control].dropna())
    results.append({'Gene names': i, 't_stat': t_stat, 'p_value': p_value})
    
# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Gene names,t_stat,p_value
0,UHRF1BP1L,-0.008397,0.993572
1,IGLC7,-0.835699,0.441417
2,SSC5D,0.773785,0.456961
3,MEGF11,-0.578341,0.584073
4,IGLON5,-1.719494,0.116267
...,...,...,...
1135,DPP3,1.165491,0.281993
1136,COX16,-0.405482,0.697230
1137,KDM5B,-0.631990,0.547471
1138,CORO1C,0.500126,0.632323


In [10]:
# Add the results DataFrame to the original data DataFrame
prot_stat = pd.concat([cont_filt, results_df.set_index('Gene names')], axis=1)
prot_stat.head()

,ALS,ALS.1,ALS.2,ALS.3,Control,ALS.4,ALS.5,ALS.6,Control.1,Control.2,ALS.7,ALS.8,Control.3,Control.4,t_stat,p_value
Gene names,,,,,,,,,,,,,,,,
UHRF1BP1L,1.549,1.083,0.907,1.284,0.925,0.647,0.620,0.794,1.109,0.673,NaN,NaN,NaN,NaN,-0.008397,0.993572
IGLC7,1.002,0.577,0.635,2.081,0.827,NaN,NaN,NaN,NaN,NaN,0.910,0.841,2.379,0.706,-0.835699,0.441417
SSC5D,1.071,1.009,1.021,0.900,0.990,1.055,1.023,0.991,1.098,0.942,1.046,0.923,0.911,0.903,0.773785,0.456961
MEGF11,1.141,0.867,0.947,0.500,1.040,0.818,1.044,0.958,1.050,0.836,NaN,NaN,NaN,NaN,-0.578341,0.584073
IGLON5,0.952,0.860,1.158,0.554,1.030,1.050,1.134,1.135,1.014,1.064,0.919,0.791,1.245,1.282,-1.719494,0.116267


In [11]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(prot_stat['p_value'], method='fdr_bh')
prot_stat['p_adjusted_BH'] = p_adjusted
prot_stat.head()


,ALS,ALS.1,ALS.2,ALS.3,Control,ALS.4,ALS.5,ALS.6,Control.1,Control.2,ALS.7,ALS.8,Control.3,Control.4,t_stat,p_value,p_adjusted_BH
Gene names,,,,,,,,,,,,,,,,,
UHRF1BP1L,1.549,1.083,0.907,1.284,0.925,0.647,0.620,0.794,1.109,0.673,NaN,NaN,NaN,NaN,-0.008397,0.993572,0.995525
IGLC7,1.002,0.577,0.635,2.081,0.827,NaN,NaN,NaN,NaN,NaN,0.910,0.841,2.379,0.706,-0.835699,0.441417,0.933089
SSC5D,1.071,1.009,1.021,0.900,0.990,1.055,1.023,0.991,1.098,0.942,1.046,0.923,0.911,0.903,0.773785,0.456961,0.933089
MEGF11,1.141,0.867,0.947,0.500,1.040,0.818,1.044,0.958,1.050,0.836,NaN,NaN,NaN,NaN,-0.578341,0.584073,0.944459
IGLON5,0.952,0.860,1.158,0.554,1.030,1.050,1.134,1.135,1.014,1.064,0.919,0.791,1.245,1.282,-1.719494,0.116267,0.816887


In [12]:
_, p_adjusted, _, _ = multipletests(prot_stat['p_value'], method='bonferroni')
prot_stat['p_adjusted_bonf'] = p_adjusted
prot_stat.head()

,ALS,ALS.1,ALS.2,ALS.3,Control,ALS.4,ALS.5,ALS.6,Control.1,Control.2,ALS.7,ALS.8,Control.3,Control.4,t_stat,p_value,p_adjusted_BH,p_adjusted_bonf
Gene names,,,,,,,,,,,,,,,,,,
UHRF1BP1L,1.549,1.083,0.907,1.284,0.925,0.647,0.620,0.794,1.109,0.673,NaN,NaN,NaN,NaN,-0.008397,0.993572,0.995525,1.0
IGLC7,1.002,0.577,0.635,2.081,0.827,NaN,NaN,NaN,NaN,NaN,0.910,0.841,2.379,0.706,-0.835699,0.441417,0.933089,1.0
SSC5D,1.071,1.009,1.021,0.900,0.990,1.055,1.023,0.991,1.098,0.942,1.046,0.923,0.911,0.903,0.773785,0.456961,0.933089,1.0
MEGF11,1.141,0.867,0.947,0.500,1.040,0.818,1.044,0.958,1.050,0.836,NaN,NaN,NaN,NaN,-0.578341,0.584073,0.944459,1.0
IGLON5,0.952,0.860,1.158,0.554,1.030,1.050,1.134,1.135,1.014,1.064,0.919,0.791,1.245,1.282,-1.719494,0.116267,0.816887,1.0


In [13]:
#df['average'] = df.mean(numeric_only=True, axis=1)
prot_stat['Mean Control'] = prot_stat.loc[:, prot_stat.columns.str.startswith('Control.')].mean(axis=1)
prot_stat['Mean ALS'] = prot_stat.loc[:, prot_stat.columns.str.startswith('ALS.')].mean(axis=1)

In [14]:
#Fold Change
prot_stat["FC ALS-C"]= prot_stat["Mean ALS"]/prot_stat["Mean Control"]

In [15]:
#Log 2 FC
prot_stat["Log2FC ALS-C"]=np.log2(prot_stat["FC ALS-C"])
prot_stat.head()

,ALS,ALS.1,ALS.2,ALS.3,Control,ALS.4,ALS.5,ALS.6,Control.1,Control.2,...,Control.3,Control.4,t_stat,p_value,p_adjusted_BH,p_adjusted_bonf,Mean Control,Mean ALS,FC ALS-C,Log2FC ALS-C
Gene names,,,,,,,,,,,,,,,,,,,,,
UHRF1BP1L,1.549,1.083,0.907,1.284,0.925,0.647,0.620,0.794,1.109,0.673,...,NaN,NaN,-0.008397,0.993572,0.995525,1.0,0.89100,0.889167,0.997942,-0.002972
IGLC7,1.002,0.577,0.635,2.081,0.827,NaN,NaN,NaN,NaN,NaN,...,2.379,0.706,-0.835699,0.441417,0.933089,1.0,1.54250,1.008800,0.654003,-0.612630
SSC5D,1.071,1.009,1.021,0.900,0.990,1.055,1.023,0.991,1.098,0.942,...,0.911,0.903,0.773785,0.456961,0.933089,1.0,0.96350,0.996000,1.033731,0.047861
MEGF11,1.141,0.867,0.947,0.500,1.040,0.818,1.044,0.958,1.050,0.836,...,NaN,NaN,-0.578341,0.584073,0.944459,1.0,0.94300,0.855667,0.907388,-0.140209
IGLON5,0.952,0.860,1.158,0.554,1.030,1.050,1.134,1.135,1.014,1.064,...,1.245,1.282,-1.719494,0.116267,0.816887,1.0,1.15125,0.950125,0.825299,-0.277012


In [17]:
prot_stat.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Biomarkers in CSF\Biomarkers T-test.xlsx")